In [1]:
from datetime import datetime, timedelta, time as dt_time
import time

class Node:
    def __init__(self, data):
        self.data = data
        self.next = None

class LinkedListQueue:
    def __init__(self):
        self.front = self.rear = None

    def enqueue(self, data):
        new_node = Node(data)
        if not self.rear:
            self.front = self.rear = new_node
        else:
            self.rear.next = new_node
            self.rear = new_node

    def dequeue(self):
        if not self.front:
            return None
        result = self.front.data
        self.front = self.front.next
        if not self.front:
            self.rear = None
        return result

    def peek(self):
        return self.front.data if self.front else None

    def is_empty(self):
        return self.front is None

    def remove(self, target):
        prev = None
        current = self.front
        while current:
            if current.data == target:
                if prev:
                    prev.next = current.next
                else:
                    self.front = current.next
                if current == self.rear:
                    self.rear = prev
                return True
            prev = current
            current = current.next
        return False

class ReservationSystem:
    def __init__(self):
        number_of_seats=100
        self.seats = {}
        self.queue = LinkedListQueue()
        self.all_seat_ids = [str(i) for i in range(1, number_of_seats+1)]

    def print_available_seats(self):
        available = [sid for sid in self.all_seat_ids if sid not in self.seats]
        print(f"[INFO] 현재 예약 가능한 좌석 {len(available)}개:")
        available_ints = set(map(int, available))
        for row in range(0, 100, 10):
            row_seats = [str(i) for i in range(row + 1, row + 11) if i in available_ints]
            print(" ".join(row_seats))

    def reserve_seat(self, seat_id, start_hour, start_minute, duration_min):
        self._clear_expired_reservations()
        if seat_id in self.seats:
            print(f"[ERROR] 좌석 {seat_id}는 이미 예약되어 있습니다.")
            return

        today = datetime.now().date()
        try:
            start_time = datetime.combine(today, dt_time(start_hour, start_minute))
        except:
            print("[ERROR] 시작 시각이 올바르지 않습니다.")
            return

        if start_minute % 10 != 0:
            print("[ERROR] 시작 분은 0, 10, 20, 30, 40, 50 중 하나여야 합니다.")
            return

        now = datetime.now()
        latest_hour = (now + timedelta(hours=2)).replace(minute=0, second=0, microsecond=0)
        if start_time > latest_hour:
            print(f"[ERROR] 예약은 현재 시각 기준 {latest_hour.strftime('%H:%M')}까지만 가능합니다.")
            return

        end_time = start_time + timedelta(minutes=duration_min)
        if end_time <= now:
            print("[ERROR] 종료 시각이 현재보다 과거입니다. 올바른 예약 시간을 입력해주세요.")
            return

        reservation = {
            'seat_id': seat_id,
            'start_time': start_time,
            'end_time': end_time
        }
        self.queue.enqueue(reservation)
        self.seats[seat_id] = reservation
        print(f"[INFO] 좌석 {seat_id}가 {start_time.strftime('%H:%M')}부터 {end_time.strftime('%H:%M')}까지 예약되었습니다.")

    def cancel_reservation(self, seat_id):
        if seat_id not in self.seats:
            print(f"[ERROR] 좌석 {seat_id}는 예약되어 있지 않습니다.")
            return
        reservation = self.seats.pop(seat_id)
        removed = self.queue.remove(reservation)
        print(f"[INFO] 좌석 {seat_id}의 예약이 취소되었습니다." if removed else "[WARN] 큐에서 해당 예약을 찾지 못했습니다.")

    def _clear_expired_reservations(self):
        now = datetime.now()
        current = self.queue.front
        prev = None
        while current:
            next_node = current.next
            data = current.data
            if data['end_time'] <= now:
                if prev:
                    prev.next = next_node
                else:
                    self.queue.front = next_node
                if current == self.queue.rear:
                    self.queue.rear = prev
                if data['seat_id'] in self.seats:
                    del self.seats[data['seat_id']]
                    print(f"[INFO] 좌석 {data['seat_id']}의 예약이 만료되어 자동 취소되었습니다.")
            else:
                prev = current
            current = next_node

    def show_status(self):
        print("\n[현재 예약 상태]")
        if not self.seats:
            print("모든 좌석이 비어 있습니다.")
        else:
            for seat_id, res in self.seats.items():
                now = datetime.now()
                if now < res['start_time']:
                    state = "(예약대기)"
                elif res['start_time'] <= now <= res['end_time']:
                    state = "(이용 중)"
                else:
                    state = "(만료 대기 중)"
                print(f"좌석 {seat_id}: {res['start_time'].strftime('%H:%M')} ~ {res['end_time'].strftime('%H:%M')} {state}")
        print()

def main():
    system = ReservationSystem()

    while True:
        system._clear_expired_reservations()
        print("==== 좌석 예약 시스템 ====")
        print("1. 좌석 예약")
        print("2. 예약 취소")
        print("3. 종료")
        choice = input("메뉴를 선택하세요: ")

        if choice == '1':
            system.print_available_seats()
            print("[INFO] 이용시작시간은 현재로부터 1시간 이내로 지정 가능합니다. ex) 현재: 08:50 -> 최대 10:00까지 지정 가능")
            seat_id = input("예약할 좌석 번호를 입력하세요: ")
            try:
                start_hour = int(input("이용시작시각(시)을 입력하세요(0~23): "))
                start_minute = int(input("이용시작시각(분)을 입력하세요(10분 단위): "))
                duration = int(input("얼마나 이용하실 건가요?(10분 단위): "))
                system.reserve_seat(seat_id, start_hour, start_minute, duration)
            except ValueError:
                print("[ERROR] 숫자로 올바르게 입력해주세요.")
        elif choice == '2':
            seat_id = input("취소할 좌석 번호를 입력하세요: ")
            system.cancel_reservation(seat_id)
        elif choice == '3':
            print("[INFO] 시스템을 종료합니다.")
            break
        else:
            print("[ERROR] 잘못된 선택입니다.")

        system.show_status()
        time.sleep(1)

if __name__ == "__main__":
    main()

==== 좌석 예약 시스템 ====
1. 좌석 예약
2. 예약 취소
3. 종료
[INFO] 현재 예약 가능한 좌석 100개:
1 2 3 4 5 6 7 8 9 10
11 12 13 14 15 16 17 18 19 20
21 22 23 24 25 26 27 28 29 30
31 32 33 34 35 36 37 38 39 40
41 42 43 44 45 46 47 48 49 50
51 52 53 54 55 56 57 58 59 60
61 62 63 64 65 66 67 68 69 70
71 72 73 74 75 76 77 78 79 80
81 82 83 84 85 86 87 88 89 90
91 92 93 94 95 96 97 98 99 100
[INFO] 이용시작시간은 현재로부터 1시간 이내로 지정 가능합니다. ex) 현재: 08:50 -> 최대 10:00까지 지정 가능
[INFO] 좌석 10가 12:10부터 12:20까지 예약되었습니다.

[현재 예약 상태]
좌석 10: 12:10 ~ 12:20 (예약대기)

==== 좌석 예약 시스템 ====
1. 좌석 예약
2. 예약 취소
3. 종료
[INFO] 현재 예약 가능한 좌석 99개:
1 2 3 4 5 6 7 8 9
11 12 13 14 15 16 17 18 19 20
21 22 23 24 25 26 27 28 29 30
31 32 33 34 35 36 37 38 39 40
41 42 43 44 45 46 47 48 49 50
51 52 53 54 55 56 57 58 59 60
61 62 63 64 65 66 67 68 69 70
71 72 73 74 75 76 77 78 79 80
81 82 83 84 85 86 87 88 89 90
91 92 93 94 95 96 97 98 99 100
[INFO] 이용시작시간은 현재로부터 1시간 이내로 지정 가능합니다. ex) 현재: 08:50 -> 최대 10:00까지 지정 가능
[INFO] 좌석 11가 12:30부터 13:00까지 예약되었습니다.

[현재 예약 상태]
좌석 

KeyboardInterrupt: Interrupted by user